In [1]:
#!pip install faiss-cpu

In [2]:
#!pip install gensim

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
import pickle
import spacy

In [5]:
from wildhack_modules.pipeline import vektors_model_fasttext, vektors_fasttext_vectorize
from wildhack_modules.pipeline import create_common_futures_learn, save_numpy_df
from wildhack_modules.pipeline import create_common_futures_lemma

In [6]:
!ls

01_read_csv.ipynb	      dataset	result		  wildhack.wpr
02-mystem.ipynb		      datasets	server_old.py	  wildhack.wpu
03_read_csv-dataset-01.ipynb  models	tmp
04-compare-quary.ipynb	      presents	wildhack_modules


In [7]:
nlp = spacy.load("ru_core_news_md")

In [8]:
df = pd.read_csv('./dataset/query_popularity.csv', sep=',')

In [9]:
#df = df[:100]

In [10]:
df

,query,query_popularity
0,ноутбук,10
1,куртка женская осенняя,10
2,ботинки женские,10
3,видеокарта,10
4,пальто женское осеннее,10
...,...,...
336983,конструктор робо-динозавры,1
336984,чика,1
336985,Дешевый товар,1
336986,макса,1


In [11]:
df['drop'] = df['query'].apply(lambda x: True if isinstance(x, str) else False)

In [12]:
df

,query,query_popularity,drop
0,ноутбук,10,True
1,куртка женская осенняя,10,True
2,ботинки женские,10,True
3,видеокарта,10,True
4,пальто женское осеннее,10,True
...,...,...,...
336983,конструктор робо-динозавры,1,True
336984,чика,1,True
336985,Дешевый товар,1,True
336986,макса,1,True


In [13]:
df = df[df['drop']==True]

In [14]:
df

,query,query_popularity,drop
0,ноутбук,10,True
1,куртка женская осенняя,10,True
2,ботинки женские,10,True
3,видеокарта,10,True
4,пальто женское осеннее,10,True
...,...,...,...
336983,конструктор робо-динозавры,1,True
336984,чика,1,True
336985,Дешевый товар,1,True
336986,макса,1,True


In [16]:
create_common_futures_learn(df, 'query')

/home/joefox/data/nextcloud/projects/wildhack/wildhack_modules/pipeline.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['learn'] = create_learn_text(df[name_column].values, {'min_token_size':0})


In [17]:
create_common_futures_lemma(df, 'learn')

/home/joefox/data/nextcloud/projects/wildhack/wildhack_modules/pipeline.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemma'] = df[name_column].apply(lambda x: [(token.lemma_) for token in nlp(x)]).str.join(' ')


In [18]:
df

,query,query_popularity,drop,learn,lemma
0,ноутбук,10,True,ноутбук,ноутбук
1,куртка женская осенняя,10,True,куртка женская осенняя,куртка женский осенний
2,ботинки женские,10,True,ботинки женские,ботинок женский
3,видеокарта,10,True,видеокарта,видеокарта
4,пальто женское осеннее,10,True,пальто женское осеннее,пальто женский осенний
...,...,...,...,...,...
336983,конструктор робо-динозавры,1,True,конструктор робо динозавры,конструктор робо динозавры
336984,чика,1,True,чика,чик
336985,Дешевый товар,1,True,дешевый товар,дешёвый товар
336986,макса,1,True,макса,макс


In [19]:
df = df.reset_index(drop = True)

In [20]:
model = vektors_model_fasttext()

In [21]:
np_vectors = vektors_fasttext_vectorize(model, df['lemma'])
np_vectors.shape

(336987, 300)

In [22]:
save_numpy_df(df[['query','query_popularity']],np_vectors,'query_popularity')